<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Lilian/European_jax_random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   7596      0 --:--:-- --:--:-- --:--:--  7596
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9 MB 19 kB/s 
     |████████████████████████████████| 1.0 MB 40.8 MB/s 
+ set +ex
Installation succeeded!


In [2]:
# Peter's code

def simple_process(key, initial_stocks, numsteps, drift, cov):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key, drift, cov, (numsteps+1,))
    sigma = jnp.diag(cov) ** 0.5
    def time_step(t, val):
        dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) + noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalue(key, initial_stocks, numsteps, drift, cov, strike):
  return jnp.mean((jnp.maximum(batch_simple(keys, initial_stocks, numsteps, drift, cov)[:,-1,:]-strike,0))) # this is assuming 1 stock for testing price (didn't take avg)

def optionvalueavg(key, initial_stocks, numsteps, drift, cov, strike):
  return jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov)[:,-1,:], axis=1)-strike,0))) # this is assuming 3 stocks in basket

In [3]:
# Peter's code

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

numstocks = 3

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
#numsteps = 10
drift = jnp.array([0.0]*numstocks)

cov = jnp.identity(numstocks)*.25*.25
initial_stocks = jnp.array([100.]*numstocks)

K = 110.0

fast_simple = jax.jit(simple_process, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, 1000000)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None))

print(initial_stocks) #S = 100
print(K) #K = 110
print(cov) #sigma = 0.25
print(drift) #drift = 0

#################################################################################### values for checking
#S, K, r, sigma, T
# 100, 110, 0, 0.25, 1
# 1 stock price should be around 6.1904
# 3 stock price should be around 2.3767
# delta should be around (0.39888 / numstocks)
####################################################################################

# option price
# 1 stock
print(optionvalue(key, initial_stocks, 1, drift, cov, K)) # numsteps here = years = 1
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, 1, drift, cov, K)) # numsteps here = years = 1

# delta test
gooptionvalue = jax.grad(optionvalue,argnums=1)
gooptionvalue(keys, initial_stocks, 1, drift, cov, K) # numsteps here = years = 1

[100. 100. 100.]
110.0
[[0.0625 0.     0.    ]
 [0.     0.0625 0.    ]
 [0.     0.     0.0625]]
[0. 0. 0.]
6.195753
2.3685586


DeviceArray([0.1331373 , 0.13304196, 0.13283278], dtype=float32)

In [4]:
def simple_process(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key, drift, cov, (numsteps+1,))
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalue(key, initial_stocks, numsteps, drift, cov, strike, T):
  return jnp.mean((jnp.maximum(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:]-strike,0))) # this is assuming 1 stock for testing price (didn't take avg)

def optionvalueavg(key, initial_stocks, numsteps, drift, cov, strike, T):
  return jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:], axis=1)-strike,0))) # this is assuming 3 stocks in basket

numstocks = 3

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 50
drift = jnp.array([0.0]*numstocks)

cov = jnp.identity(numstocks)*.25*.25
initial_stocks = jnp.array([100.]*numstocks)

T = 1.0
K = 110.0

fast_simple = jax.jit(simple_process, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, 1000000)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# print(initial_stocks) #S = 100
# print(K) #K = 110
# print(cov) #sigma = 0.25
# print(drift) #drift = 0

#################################################################################### values for checking
#S, K, r, sigma, T
# 100, 110, 0, 0.25, 1
# 1 stock price should be around 6.1904
# 3 stock price should be around 2.3767
# delta should be around (0.39888 / numstocks)
####################################################################################

# option price
# 1 stock
# print(optionvalue(key, initial_stocks, numsteps, drift, cov, K, T)) # numsteps here = 50, but T = year = 1
# 3 stocks basket
# print(optionvalueavg(key, initial_stocks, numsteps, drift, cov, K, T)) # numsteps here = 50, but T = year = 1

# delta test
gooptionvalue = jax.grad(optionvalue,argnums=1)
gooptionvalue(keys, initial_stocks, numsteps, drift, cov, K, T) # numsteps here = 50, but T = year = 1

DeviceArray([0.1331037 , 0.13305795, 0.13296409], dtype=float32)

# Random Test

In [60]:
numstocks = 2  # dimension of multivariate normal distribution
numsteps = 10000   # length of each random variable
numpath = 10  # number of split

drift = jnp.array([0.0]*numstocks)
cov = jnp.identity(numstocks)*.25*.25

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
keys = jax.random.split(key, numpath)
random_num = jax.random.multivariate_normal(keys[0], drift, cov, (numsteps+1,)).reshape((numstocks,(numsteps+1)))

indx = 0
for i in range(1,numpath):
  new_random_num = jax.random.multivariate_normal(keys[i], drift, cov, (numsteps+1,)).reshape((numstocks,(numsteps+1)))
  # random_num = jax.ops.index_update(random_num,
  #                 jax.ops.index[indx],
  #                           new_random_num)
  random_num = jax.numpy.append(random_num, new_random_num, axis=0)
  indx += 1

correlation

In [61]:
jnp.corrcoef(random_num)

DeviceArray([[ 9.99999940e-01, -1.66527892e-03, -2.73333862e-03,
              -4.33725677e-03, -5.46932314e-03, -1.17343664e-02,
              -4.47496917e-04,  7.34949531e-03, -7.90107809e-03,
              -7.27269007e-03, -8.33620038e-03,  6.53386628e-03,
              -1.02988947e-02,  5.14707062e-03,  1.22996299e-02,
               2.30878592e-03,  7.87848420e-03, -2.74084019e-03,
              -7.02941092e-04, -5.82124479e-03],
             [-1.66527880e-03,  1.00000000e+00,  1.63166374e-02,
              -8.36731866e-03,  1.59537829e-02, -1.52031314e-02,
               4.50421451e-03,  4.69492923e-04,  1.28774056e-02,
               5.48884925e-03, -2.96503510e-02, -1.12845311e-02,
               9.98777524e-03,  1.30028492e-02, -8.45674518e-03,
              -1.01837413e-02,  1.17404833e-02, -2.93281605e-03,
               1.81764737e-03, -1.20112691e-02],
             [-2.73333886e-03,  1.63166355e-02,  1.00000000e+00,
              -1.14374550e-03,  2.19265092e-03,  8.696722

average

In [62]:
for i in random_num:
  print(jnp.average(i)) 

0.0009476914
0.0023608753
0.0031617624
0.0017355053
-0.00041941102
0.0048201764
-0.0009647168
0.00023759922
0.0024664337
-0.00071392045
0.0029935006
-0.0041032676
-3.887732e-05
0.0028897254
-0.0025956875
-0.0040349597
0.00336093
-0.00041321976
-0.0026797494
0.0015791528


standard error

In [64]:
for i in random_num:
  print(jnp.std(i)) 

0.24958338
0.25063002
0.25090125
0.24920103
0.24780434
0.25262636
0.2499293
0.25019038
0.24806991
0.25028837
0.25036404
0.25165486
0.24980706
0.2504593
0.24866036
0.24909538
0.25149482
0.24659039
0.24966395
0.24740894
